<a href="https://colab.research.google.com/github/marketpsych/marketpsych/blob/main/notebooks/load_data_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading CSV files directly from SFTP

This notebook shows how to load MarketPsych's data with your SFTP credentials directly into a Jupyter Notebook. Note, however, that this notebook is an **alpha version**. For more robust testing, we recommend downloading the trialing flat files, as instructed by the MRNSupport, and then loading them into your own environment. 

---
## Settings
In order to have this example working, you'll need to install MarketPsych's library. Additionally, this notebook uses some widgets to facilitate with your navigation. To install the libraries and enable widgets, please run the following cell. 

In [1]:
import sys
# Installs marketpsych library into your environment
!{sys.executable} -m pip install marketpsych --upgrade --quiet
# Installs ipywidget library into your environment
!{sys.executable} -m pip install ipywidgets --upgrade --quiet

## Libraries
from marketpsych import sftp
from marketpsych import mpwidgets

# Allows using the widgets
!{sys.executable} -m jupyter nbextension enable --py widgetsnbextension

from IPython.core.magic import register_cell_magic
from IPython.display import HTML, display

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


---
## Selecting your login credentials

Please input your credentials, i.e., the path to the `Key file`, and `UserID` as provided by the MRNSupport. 

<font color='blue'>**IMPORTANT**</font>       
<font color='blue'>1. Run the following cell. After running it, you should see two widgets.</font>  
<font color='blue'>2. Click on the uploader named `Key File`.</font>        
<font color='blue'>3. Check if your `User ID` is correct, if not, change it manually.</font>  
<font color='blue'>4. Move on to the next cell.</font>  

In [2]:
cwdgts = mpwidgets.LoginWidgets()
cwdgts.display()

**WARNING**                         
When you run the following cell after waiting for too long, you may be given the following error:
```python
"ValueError": I/O operation on closed file.
```   

If so, please re-run the previous cell and upload your key. Otherwise, continue.

In [3]:
# Creates client
client = sftp.connect(user=cwdgts.id_widget.value, key=cwdgts.key_widget.content)

---
## Loading the data

Finally, you can download the files directly into a pandas dataframe. The options can be defined through the 5 widgets below.

 - Select the checkbox if you are trialing the data. In some special cases, even if you are trialing, you may need to uncheck it (you can try both options in case of Permission errors).

 - The options for **Asset class** are:  

|Asset class   | Description|
|:-------------|:------------|
|`CMPNY`       | Individual companies|
|`CMPNY_AMER`  | Individual companies domiciled in America|
|`CMPNY_APAC`  | Individual companies domiciled in APAC|
|`CMPNY_EMEA`  | Individual companies domiciled in EMEA|
|`CMPNY_ESG`   | Individual companies (ESG package)|
|`CMPNY_GRP`   | Company groups and ETFs|
|`COM_AGR`     | Agricultural commodities|  
|`COM_ENM`     | Energy and Metals|
|`COU`         | Countries|
|`COU_ESG`     | Countries (ESG package)|
|`COU_MKT`     | Stock indices, sovereign bonds, real estate|
|`CRYPTO`      | Cryptocurrencies|
|`CUR`         | Currencies|  

 - The options for **frequencies** are:  
  
|Frequency  | Description| Use case |
|:----------|:-----------|:---------|
|`W365_UDAI`| Yearly lookback window and daily updates| ESG Core only |
|`WDAI_UDAI`| Daily lookback window and daily updates| Daily data stamped 30 minutes before the NYSE close|
|`WDAI_UHOU`| Daily lookback window and hourly updates| Daily data stamped hourly (in case you want daily data adjusted to your time-zone) |
|`W01M_U01M`| Minutely lookback window and minutely updates| Low-latency data (**WARNING:** extremely large datasets)|
 
- Regarding **dates**, simply select the start and end dates of interest. Note that for dates older than 2 months (with respect to your current date), the files are packaged in monthly batches. Hence, only the selection of month will matter and not the specific day. For example, if you select `2020-12-25` as the start or end date, the full `2020-12` month will be loaded anyways.

**WARNINGS**   
Loading large files such as CMPNY data with a long window-frame can take quite a while and take over your memory. Start by loading very short periods (e.g., one month of data), then moving to ever longer periods. 

Check your asset class permissions. If you try downloading data for which no access was provided, it will give a Permission error:
```python
"PermissionError": [Errno 13] Access denied
```   

<font color='blue'>Select options for loading the data after running the cell:</font> 

In [4]:
lwdgts = mpwidgets.LoaderWidgets()
lwdgts.display()

Checkbox(value=True, description='Trial')

Dropdown(description='Asset class:', index=7, options=('CMPNY', 'CMPNY_AMER', 'CMPNY_APAC', 'CMPNY_EMEA', 'CMP…

Dropdown(description='Frequency:', index=1, options=('W365_UDAI', 'WDAI_UDAI', 'WDAI_UHOU', 'W01M_U01M'), valu…

DatePicker(value=datetime.datetime(2020, 12, 1, 0, 0), description='Start date:')

DatePicker(value=datetime.datetime(2020, 12, 31, 0, 0), description='End date:')

Output()

In [5]:
%%time

df = client.download(
    asset_class=sftp.AssetClass[lwdgts.asset_class_widget.value],
    frequency=sftp.Frequency[lwdgts.frequency_widget.value],
    start=lwdgts.start_date_widget.value,
    end=lwdgts.end_date_widget.value,
    trial=lwdgts.trial_check_widget.value
)

display(df)

,id,assetCode,windowTimestamp,dataType,systemVersion,mentions,buzz,sentiment,negative,positive,...,overvaluedVsUndervalued,volatility,consumptionVolume,productionVolume,regulatoryIssues,supplyVsDemand,supplyVsDemandForecast,newExploration,safetyAccident,futureVsPast
0,mp:2020-12-01_20.30.00.News.COM_ENM.ALU,ALU,2020-12-01T20:30:00.000Z,News,MP:4.0.0,261,1036.6,0.101775,0.188115,0.289890,...,NaN,0.045823,NaN,-0.001929,NaN,-0.013506,-0.005788,0.000965,NaN,-0.181266
1,mp:2020-12-01_20.30.00.News_Social.COM_ENM.ALU,ALU,2020-12-01T20:30:00.000Z,News_Social,MP:4.0.0,337,1253.7,0.099705,0.180266,0.279971,...,NaN,0.043471,NaN,0.000000,NaN,-0.003191,-0.003191,0.000798,NaN,-0.158650
2,mp:2020-12-01_20.30.00.News_Headline.COM_ENM.ALU,ALU,2020-12-01T20:30:00.000Z,News_Headline,MP:4.0.0,15,22.0,0.272727,0.181818,0.454545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.363636
3,mp:2020-12-01_20.30.00.Social.COM_ENM.ALU,ALU,2020-12-01T20:30:00.000Z,Social,MP:4.0.0,76,217.1,0.089820,0.142791,0.232612,...,NaN,0.032243,NaN,0.009212,NaN,0.046062,0.009212,NaN,NaN,-0.050668
4,mp:2020-12-01_20.30.00.News.COM_ENM.ANGS,ANGS,2020-12-01T20:30:00.000Z,News,MP:4.0.0,19,110.0,0.254545,0.045455,0.300000,...,NaN,NaN,0.018182,NaN,NaN,-0.009091,NaN,NaN,NaN,-0.327273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,mp:2020-12-31_20.30.00.Social.COM_ENM.USCRU,USCRU,2020-12-31T20:30:00.000Z,Social,MP:4.0.0,1118,4281.6,0.084081,0.180306,0.264387,...,-0.001868,0.015415,0.000467,-0.000934,NaN,0.003970,-0.004905,0.000467,NaN,-0.083520
4460,mp:2020-12-31_20.30.00.News.COM_ENM.ZNC,ZNC,2020-12-31T20:30:00.000Z,News,MP:4.0.0,82,267.7,0.220396,0.115801,0.336197,...,NaN,0.011207,NaN,0.014942,NaN,0.014942,0.022413,0.007471,NaN,-0.222264
4461,mp:2020-12-31_20.30.00.News_Social.COM_ENM.ZNC,ZNC,2020-12-31T20:30:00.000Z,News_Social,MP:4.0.0,148,444.9,0.168577,0.148348,0.316925,...,NaN,0.010115,NaN,0.008991,NaN,0.008991,0.013486,0.004495,NaN,-0.102270
4462,mp:2020-12-31_20.30.00.News_Headline.COM_ENM.ZNC,ZNC,2020-12-31T20:30:00.000Z,News_Headline,MP:4.0.0,2,6.0,0.333333,NaN,0.333333,...,NaN,NaN,NaN,0.333333,NaN,0.333333,0.333333,NaN,NaN,1.000000


CPU times: user 140 ms, sys: 31.6 ms, total: 171 ms
Wall time: 8.72 s


If you can see a dataframe above, congratulations! You have downloaded some data into your notebook. From here, you can have fun exploring it. Below, you'll find a plotting tool for some simple understanding of the data.

---
## Visualizing the data

Below you can use the widgets to do some very basic exploration. A description of the widgets is given below.

- The **Data Type** field represents the type of content source(s) on which the RMAs are based. There are four possible values:
    - `News` for news sources (headlines and corpus)
    - `News_Headline` for the headline only of news sources
    - `Social` for social media sources
    - `News_Social` for the combined content               


- The **Anaytics** field represents the RMA. The actual values will depend on the asset class. Several types of indicators are provided:
    - Emotional indicators such as Anger, Fear and Joy
    - 'Economic' metrics including Earnings Forecast, Interest Rate Forecast, Long vs. Short 
    - ESG measures including CarbonEmissionsControversy, ManagementTrust, and WorkplaceSafety
    - etc. 


- The **Asset** field represents the asset of choice. To see all options, clear the cell. For a description of each asset, please search for the asset code in the User Guide or Eikon app.


- The **Roll. window** field represents the length in the smoothing function (a simple moving average).

The indicators are updated every minute for companies, sectors, regions, countries, commodities, indices, bonds, currencies and cryptocurrencies. They can be translated directly into spreadsheets or charts that can be monitored by traders, risk managers or analysts – or they can be plugged straight into your algorithms for low frequency or longer-term asset allocation or sector rotation decisions.

**WARNING**                
If your plot is empty, it is likely that there is no data for that combination of the three top variables. 

<font color='blue'>Select options for plotting the data after running the cell:</font> 

In [6]:
swdgts = mpwidgets.SlicerWidgets(df)
swdgts.display()